# Planning-Lab Lesson 2: Informed Search Strategies

In the second session we will work on informed search

### Maze Environments
The environments used is **SmallMaze** (visible in the figure).

<img src="images/maze.png" width="300">

The agent starts in cell $(0, 2)$ and has to reach the treasure in $(4, 3)$.

### Priority Queue

You will need a queue ordered by priority as a queue or **PriorityQueue**. The difference between the other versions of the queue is that in **PriorityQueue**, nodes are removed from the data structure based on the current lowest value. In particular, **Node** has two useful parameters (other than those used in the previous session):

- pathcost - the path cost from the root node to the current one (defaults to 0)
- value - the value of a node. Used by PriorityQueue to order its content (defaults to 0)

### Here is an example of usage:

In [32]:
import os
import sys
module_path = os.path.abspath(os.path.join('../tools'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.ai_lab_functions import *

# Create 3 nodes for state ids 1 2 3
node_1 = Node(1) # No parent, pathcost=0, value=0
node_2 = Node(2, node_1, node_1.pathcost + 1, 10) # Child of node_1, pathcost=1, value=10
node_3 = Node(3, node_1, 100, 5)  # Child of node_1, pathcost=100, value=5

p_queue = PriorityQueue()
for n in (node_1, node_2, node_3):
    p_queue.add(n)
    print("Added: {}".format(n.state))

while not p_queue.is_empty():
    node = p_queue.remove()
    print("Removed state : {}, path cost: {}, value: {}".format(node.state,node.pathcost,node.value))

Added: 1
Added: 2
Added: 3
Removed state : 1, path cost: 0, value: 0
Removed state : 3, path cost: 100, value: 5
Removed state : 2, path cost: 1, value: 10


Notice the order in which nodes are removed from the queue.

## Uniform-Cost Search (UCS)
Informed strategy can be considered as a specific implementation of Uniform-Cost Search (UCS). The following code implements a UCS, *graph search* version. The cost of performing an action is supposed to be always 1 (also in the assignments).

In [33]:
def present_with_higher_cost(queue, node):
    if node.state in queue:
        if queue[node.state].pathcost > node.pathcost: 
            return True
    return False

In [34]:
import gym
import envs

def ucs(environment):
    """
    Uniform-cost search
    
    Args:
        environment: OpenAI Gym environment
        
    Returns:
        path: solution as a path
    """
    
    queue = PriorityQueue()
    queue.add(Node(environment.startstate))
    
    explored = set()
    time_cost = 1
    space_cost = 1
    
    while True:
        if queue.is_empty(): 
            return None, time_cost, space_cost 
        
        # Retrieve node from the queue
        node = queue.remove()  
        if node.state == environment.goalstate: 
            return build_path(node), time_cost, space_cost
        
        explored.add(node.state)
        
        # Look around
        for action in range(environment.action_space.n):
            
            # Child node where value and pathcost are both the pathcost of parent + 1
            child = Node(environment.sample(node.state, action), node, node.pathcost + 1, node.pathcost + 1)  
            time_cost += 1
            
            if child.state not in queue and child.state not in explored:
                queue.add(child)
                
            elif present_with_higher_cost(queue, child):
                queue.replace(child)
                
        space_cost = max(space_cost, len(queue) + len(explored))


#### Let's see the results:

In [35]:
# Create and render the environment
env = gym.make("SmallMaze-v0")
env.render()
solution, time, memory = ucs(env)

CheckResult_UCS(solution, time, memory, env)

[['C' 'C' 'S' 'C']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'W' 'W' 'W']
 ['C' 'C' 'C' 'G']]

##########################################
#####  UNIFORM GRAPH SEARCH PROBLEM  #####
##########################################
Solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 61
Max n° of nodes in memory: 16


## Distance Heuristics

Informed search requires a distance heuristic to estimate the distance between a state and the goal. You already have at your disposal these functions:

- *l1_norm(p1, p2)* - Computes the L1 norm (also known as the manhattan distance) between two points specified as tuples of coordinates.
- *l2_norm(p1, p2)* - Computes the L2 norm between two points specified as tuples of coordinates.
- *chebyshev(p1, p2)* - Computes the Chebyshev distance between two points specified as tuples of coordinates. Similar to the L1 norm but diagonal moves are also considered.


**Examples:**

In [36]:
p1 = (0, 2)
p2 = (4, 0)
print("L1 norm heuristic value: {}".format(Heu.l1_norm(p1, p2)))
print("L2 norm heuristic value: {}".format(Heu.l2_norm(p1, p2)))
print("Chebyshev heuristic value: {}".format(Heu.chebyshev(p1, p2)))

L1 norm heuristic value: 6
L2 norm heuristic value: 4.47213595499958
Chebyshev heuristic value: 4


# Assignment 1: Greedy Best-First Search

The first assignment is to implement the Greedy-best-first search algorithm on **SmallMaze**. In particular, you have to implement both *greedy_tree_search* and *greedy_graph_search* versions that will be called by the generic *greedy*. Use the L1 norm as a heuristic function first, then try the others to see the differences.

The results returned by greedy must be in the following form (path, time_cost, space_cost), more in detail:

- **path** - a tuple of state identifiers forming a path from the start state to the goal state. None if no solution is found.
- **time_cost** - the number of nodes checked during the exploration.
- **space_cost** - the maximum number of nodes in memory simultaneously.


### Functions to implement:
- *greedy_tree_search(environment)*
- *greedy_graph_search(environment)*

**The following function is a revised version of the present_with_higher_cost function that now checks for the value of the node, not the path cost. This is the version you should use in the graph search implementation of greedy and A***

In [37]:
def present_with_higher_value(queue, node):
    if node.state in queue:
        if queue[node.state].value > node.value: 
            return True
    return False

In [38]:
def greedy_tree_search(environment, timeout=10000):
    """
    Greedy-best-first Tree search
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """

    goalpos = environment.state_to_pos(environment.goalstate)
    queue = PriorityQueue()
    #
    # YOUR CODE HERE ...
    #
    time_cost = 1
    space_cost = 1
    #
    # YOUR CODE HERE ...
    #
    while True:
        if time_cost >= timeout: return ("time-out", time_cost, space_cost) # timeout check
        #
        # YOUR CODE HERE ...
        #
        return None, time_cost, space_cost

In [39]:
def greedy_graph_search(environment):
    """
    Greedy-best-first Graph search
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    #
    # YOUR CODE HERE ...
    #
    explored = set()
    time_cost = 1
    space_cost = 1
    #
    # YOUR CODE HERE ...
    #
    return None, time_cost, space_cost

**The following function calls your implementations of greedy_tree_search and greedy_graph_search:**

In [40]:
def greedy(environment, search_type):
    """
    Greedy-best-first search
    
    Args:
        problem: OpenAI Gym environment
        search_type: type of search - greedy_tree_search or greedy_graph_search (function pointer)
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    path, time_cost, space_cost = search_type(environment)
    return path, time_cost, space_cost

**The following code calls your *tree search* and *graph search* version of Greedy-best-first search and checks the results:**

In [41]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts = greedy(environment, greedy_tree_search)
solution_gs, time_gs, memory_gs = greedy(environment, greedy_graph_search)
heuristic = str(input('Which heuristic did you use? (l1_norm, l2_norm, chebyshev)\n'))

results = CheckResult_L2A1([solution_ts, time_ts, memory_ts], [solution_gs, time_gs, memory_gs], heuristic, env)
results.check_sol_ts()
results.check_sol_gs()

Which heuristic did you use? (l1_norm, l2_norm, chebyshev)
l1_norm
########################################################
#######  GREEDY BEST FIRST TREE SEARCH PROBLEM  ########
########################################################
Your solution: None
N° of nodes explored: 1
Max n° of nodes in memory: 1

> Your solution is not correct!

########################################################
#######  GREEDY BEST FIRST GRAPH SEARCH PROBLEM  #######
########################################################
Your solution: None
N° of nodes explored: 1
Max n° of nodes in memory: 1

> Your solution is not correct, should be: [(0, 3), (1, 3), (2, 3), (2, 2), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]



# Assignment 2: A* Search
The second assignment is to implement the A* search algorithm on SmallMaze. In particular, you have to implement both astar_tree_search and astar_graph_search versions that the generic astar will call. Use the L1 norm as a heuristic function first, then try the others to see the differences.

The results returned by astar must be in the following form (path, time_cost, space_cost), more in detail:

- **path** - a tuple of state identifiers forming a path from the start state to the goal state. None if no solution is found.
- **time_cost** - the number of nodes checked during the exploration.
- **space_cost** - the maximum number of nodes in memory simultaneously.

Functions to implement:
- *astar_tree_search(environment)*
- *astar_graph_search(environment)*

In [42]:
def astar_tree_search(environment):
    """
    A* Tree search
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """

    goalpos = environment.state_to_pos(environment.goalstate)
    queue = PriorityQueue()
    #
    # YOUR CODE HERE ...
    #    
    time_cost = 1
    space_cost = 1
    #
    # YOUR CODE HERE ...
    #
    return None, time_cost, space_cost

In [43]:
def astar_graph_search(environment):
    """
    A* Graph Search
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    #
    # YOUR CODE HERE ...
    #
    explored = set()
    time_cost = 1
    space_cost = 1
    #
    # YOUR CODE HERE ...
    #
    return None, time_cost, space_cost

**The following function calls your implementations of astar_tree_search and astar_graph_search:**

In [44]:
def astar(environment, search_type):
    """
    A* search
    
    Args:
        environment: OpenAI Gym environment
        search_type: type of search - astar_tree_search or astar_graph_search (function pointer)
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    path, time_cost, space_cost = search_type(environment)
    return path, time_cost, space_cost

**The following code calls your *tree search* and *graph search* version of A* search and checks the results:**

In [45]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts = astar(environment, astar_tree_search)
solution_gs, time_gs, memory_gs = astar(environment, astar_graph_search)
heuristic = str(input('Which heuristic did you use? (l1_norm, l2_norm, chebyshev)\n'))

results = CheckResult_L2A2([solution_ts, time_ts, memory_ts], [solution_gs, time_gs, memory_gs], heuristic, env)
results.check_sol_ts()
results.check_sol_gs()

Which heuristic did you use? (l1_norm, l2_norm, chebyshev)
l1_norm
#########################################
#######  A* TREE SEARCH PROBLEM  ########
#########################################
Your solution: None
N° of nodes explored: 1
Max n° of nodes in memory: 1

> Your solution is not correct, should be: [(0, 1), (1, 1), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]

##########################################
#######  A* GRAPH SEARCH PROBLEM  ########
##########################################
Your solution: None
N° of nodes explored: 1
Max n° of nodes in memory: 1

> Your solution is not correct, should be: [(0, 1), (1, 1), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]



### Discussion
Now that you have correctly implemented both Greedy-best-first and A* what can you say about the solutions they compute? Are there significant differences in the stats? Try to play with other heuristics as well and see if your results change.